#1 Declare utility functions for data visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def display_images_in_table(img_set, label_set, color_map=None, max_columns=3, figsize=(20, 14), show_axes=True) :
    '''
    Display provided array of images in a form of the table.
    Signs each displayed image with corresponding label from provided arry of string labels
    img_set - array of images to display
    label_set - array of string labels to sign the images. Note!: Must be same size as array of images
    color_map - color map to use for visualization. If not provided - RGB will be used
    '''
    assert(len(img_set) == len(label_set))
    rows = (len(label_set) / max_columns) + 1
    plt.figure(figsize=figsize)
    for i in range(len(img_set)):
        label = label_set[i]
        # Each subplot is an image with its label    
        a = plt.subplot(rows, max_columns, i + 1)
        plt.title(label)
        # get correspondent image from image set 
        if (len(img_set[i].shape) == 1):
            image = np.ones((np.amax(img_set[i] * 6), img_set[i].shape[0]), dtype=np.int32)
            y = list(map(lambda x:6*x, img_set[i]))
            for j in range(y.shape[0]):
                image[y[j], j] = 0
        else:
            image = img_set[i].squeeze()   
        fig = plt.imshow(image, cmap='gray')
        if(not show_axes):
            # hide image size scale axes
            fig.axes.get_xaxis().set_visible(False)
            fig.axes.get_yaxis().set_visible(False) 
        
    plt.show() 
    
def display_image_and_plot(image, plot, image_label, plot_label):
    plt.figure(figsize=(10, 3))
    plt.subplot(1, 2, 1)
    fig = plt.imshow(image)
    plt.title(image_label)
    plt.subplot(1, 2, 2)
    fig = plt.plot(plot)
    plt.title(plot_label)
    plt.show()

def display_dataset_samples(n_images_per_class, cars, notcars):
    sample_cars = []
    sample_car_labels = []
    sample_notcars = []
    sample_notcar_labels = []
    for i in range(n_images_per_class):
        car_file = cars[np.random.randint(len(cars))]
        sample_cars.append(mpimg.imread(car_file))
        sample_car_labels.append(car_file.split("/")[-1])
        
        notcar_file = notcars[np.random.randint(len(notcars))]
        sample_notcars.append(mpimg.imread(notcar_file))
        sample_notcar_labels.append(notcar_file.split("/")[-1])
    print("Random {} 'car' images from dataset:".format(n_images_per_class))    
    display_images_in_table(sample_cars, sample_car_labels, max_columns=5, figsize=(20,10), show_axes=False)   
    print("Random {} 'not a car' images from dataset:".format(n_images_per_class))    
    display_images_in_table(sample_notcars, sample_notcar_labels, max_columns=5, figsize=(20,10), show_axes=False)
    
def display_spatial_binning_of_color(image_file):
    image = mpimg.imread(image_file)
    if(image_file.endswith(".png")): 
            image = (image*255).astype(np.uint8)
    # call feature vector extraction function        
    feature_vec = bin_spatial(image, size=(32, 32))
    # show results
    display_image_and_plot(image, feature_vec, "Original Image", "Spatially Binned Color channels")

def display_histograms_of_colors(image_file):
    image = mpimg.imread(image_file)
    if(image_file.endswith(".png")): 
            image = (image*255).astype(np.uint8)
    # call feature vector extraction function      
    r_hist, g_hist, b_hist, bin_centers, feature_vec = color_hist(image, nbins=64, bins_range=(0, 256), vis=True)
    # show results
    fig = plt.figure(figsize=(12,3))
    plt.subplot(141)
    fig = plt.imshow(image)
    plt.title("Original Image")
    plt.subplot(142)
    plt.bar(bin_centers, r_hist[0])
    plt.xlim(0, 256)
    plt.title('R channel histogram')
    plt.subplot(143)
    plt.bar(bin_centers, g_hist[0])
    plt.xlim(0, 256)
    plt.title('G channel histogram')
    plt.subplot(144)
    plt.bar(bin_centers, b_hist[0])
    plt.xlim(0, 256)
    plt.title('B channel histogram')
    plt.show()
    
def display_histograms_of_gradients(image_file):
    image = mpimg.imread(image_file)
    if(image_file.endswith(".png")): 
            image = (image*255).astype(np.uint8)
    grayscale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # dummy HOG parameters
    orient = 8
    pix_per_cell = 8
    cell_per_block = 5
    # call feature vector extraction function 
    features, hog_image = get_hog_features(grayscale, orient, 
                        pix_per_cell, cell_per_block, 
                        vis=True, feature_vec=False)
    # show results
    display_images_in_table([image, hog_image], ["Original image","Histogram of Gradients"],figsize=(12,5))


#2 Load the dataset

In [ ]:
import glob

car_images = glob.glob('./dataset/vehicles/**/*.png', recursive=True)
cars = []
for image_file_path in car_images:
    cars.append(image_file_path)
print("Number of images with vehicles: ", len(cars))     
    
notcar_images = glob.glob('./dataset/non-vehicles/**/*.png', recursive=True)
notcars = []    
for image_file_path in notcar_images:
    notcars.append(image_file_path)
print("Number of images without vehicles: ", len(notcars))

# visualization of  random images from both 'car'and 'not car' datasets (10 images each)
display_dataset_samples(10, cars, notcars)

#3 Generating feature vector from images using Spatial Binning of color:

In [ ]:
import cv2

def bin_spatial(img, size=(32, 32)):
    '''
    Returns the vector of featues extracted from given image using Spatial Binnig of Color channels
    img - image to extract features
    size - target size for imge resizing
    '''
    # decrease the image resolution (separately ech color channel) by resizing to given size
    # and flatten resulting imges to a 1D feature vector
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    # combine generated feature fectors into single vector
    return np.hstack((color1, color2, color3))

print("Feature vector generated using Spatial Binning of Color channles of random 'car' image")
display_spatial_binning_of_color(cars[np.random.randint(len(cars))])
print("Feature vector generated using patial Binning of Color channles of random 'not car' image")
display_spatial_binning_of_color(notcars[np.random.randint(len(notcars))])

# global configuration parameter for Spatial Binning of Colors based feature set extration for training data generation
SPATIAL_SIZE = (32, 32) # Spatial binning dimensions

#4 Generate feature vector from images using Histogram of Color channels

In [ ]:
def color_hist(img, nbins=32, bins_range=(0, 256), vis=False):
    '''
    Returns the vector of featues extracted from given image using Color Histogram method
    img - image to extract features
    nbins - number of bins to split the pixel values of each color channel
    '''
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    
    if(vis == True):
           # Generating bin centers
        bin_edges = channel1_hist[1]
        bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
        # Concatenate the histograms into a single feature vector
        return channel1_hist, channel2_hist, channel3_hist, bin_centers, hist_features
    else:
        return hist_features 

print("Histograms of color channles of random 'car' image")
display_histograms_of_colors(cars[np.random.randint(len(cars))])
print("Histograms of color channles of random 'not car' image")
display_histograms_of_colors(notcars[np.random.randint(len(notcars))])

# global configuration parameter for Histogram of Color based feature set xstarction for training data generation
N_HISTOGRAM_BINS = 32    # Number of histogram bins

#5 Generate feature vector from images using Histogram of Gradients

In [ ]:
from skimage.feature import hog

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    '''
    Extract the features from given images generated using Histogram Of Gradients method
    
    img is image to featch the feature vector
    orient is a number of orientation bins
    pix_per_cell is a number of pixels per cell (e.g. 32 means a suare cell of 32x32 pixels)
    cell_per_block is number of cells per block (e.g 8 means a square block of 8x8 cells)
    vis is a boolean flag declaring whether the visualization image of extracted features should be returned as well
    feature_vec is a boolean flag declaring whether array of features should beflattened into the 1D vector
    '''
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features   
    
print("Histogram of gradients of random 'car' image")
display_histograms_of_gradients(cars[np.random.randint(len(cars))])
print("Histogram of gradients of random 'not car' image")
display_histograms_of_gradients(notcars[np.random.randint(len(notcars))])  

# global configuration parameters for hog based feture set extraction of training data preparat
COLOR_SPACE = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
N_ORIENTATIONS = 12 # HOG orientations
N_PIXELS_PER_CELL = 8 # HOG pixels per cell
N_CELLS_PER_BLOCK = 2 # HOG cells per block
HOG_CHANNEL = "ALL" # Can be 0, 1, 2, or "ALL"

#6 Generate a training and test feature and lebel sets from loaded dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

def change_color_space(rgb_image, color_space='RGB'):
    '''
    Changes the color space of given RGB image to provided color space
    rgb_image - image to convert. should be provided in 0-255 pixel value scale
    color_space - target convertion color space
    '''
    if color_space != 'RGB':
        if color_space == 'HSV':
            return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2YCrCb)
    else: return np.copy(rgb_image) 


def extract_features(image_files, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # check the dataset size
    assert(len(image_files) > 0)
    # Resulting list to populate with feature vectors
    features = []
    # Iterate through the list of images
    for image_file in image_files:
        image_features = []
        # Read the emage
        image = mpimg.imread(image_file)
        # rescale pixel values to [0:255] if image is PNG (matplot read ong in [0:1] pixel value scale)
        if(image_file.endswith(".png")): 
            image = (image*255).astype(np.uint8)
        # apply conversion of image to target color space
        image = change_color_space(image, color_space)      
        
        if spatial_feat == True:
            # genereta feature fector using Spatial Binning of Colors method 
            spatial_features = bin_spatial(image, size=spatial_size)
            image_features.append(spatial_features)
        
        if hist_feat == True:
            # genereta feature fector using Histogram of Colors method 
            hist_features = color_hist(image, nbins=hist_bins)
            image_features.append(hist_features)
        
        if hog_feat == True:
            # generate feature fector using Histogram of Gradients method 
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(image.shape[2]):
                    hog_features.append(get_hog_features(image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            image_features.append(hog_features)
        
        features.append(np.concatenate(image_features))
        # print the sizeof generated feature vectors (only for first image in a dataset, since all other will be teh same)
        if(image_file == image_files[0]):
            print("... spatial features vector size: ", spatial_features.shape)
            print("... color histogram features vector size: ", hist_features.shape)
            print("... HOG features vector size (multichannel): ", hog_features.shape)
            
    print("... total feature vector size: ", features[0].shape)
    # Return list of feature vectors
    return features



SPATIAL_BINNING_FEATURES_ON = True # Spatial features on or off
COLOR_HISTOGRAM_FEATURES_ON = True # Histogram features on or off
HOG_FEATURES_ON = True # HOG features on or off
print("Feature set extraction params:") 
if (HOG_FEATURES_ON):
    print("...HOG: Number of orientation bins: ", N_ORIENTATIONS)
    print("...HOG: Pixels per cell: {}x{}".format(N_PIXELS_PER_CELL, N_PIXELS_PER_CELL))
    print("...HOG: Cells per block: {}x{}".format(N_CELLS_PER_BLOCK, N_CELLS_PER_BLOCK))

if(SPATIAL_BINNING_FEATURES_ON):
    print("...Spatial Binning of Colors: binning size: ",  SPATIAL_SIZE)

if(COLOR_HISTOGRAM_FEATURES_ON):      
    print("...Histogram of Colors: number of histogram bins ",  N_HISTOGRAM_BINS)

print("Fetching feature set for 'car' images...")
car_features = extract_features(cars, color_space=COLOR_SPACE, 
                        spatial_size=SPATIAL_SIZE, hist_bins=N_HISTOGRAM_BINS, 
                        orient=N_ORIENTATIONS, pix_per_cell=N_PIXELS_PER_CELL, 
                        cell_per_block=N_CELLS_PER_BLOCK, 
                        hog_channel=HOG_CHANNEL, spatial_feat=SPATIAL_BINNING_FEATURES_ON, 
                        hist_feat=COLOR_HISTOGRAM_FEATURES_ON, hog_feat=HOG_FEATURES_ON)

print("Fetching feature set for 'not car' images...")
notcar_features = extract_features(notcars, color_space=COLOR_SPACE, 
                        spatial_size=SPATIAL_SIZE, hist_bins=N_HISTOGRAM_BINS, 
                        orient=N_ORIENTATIONS, pix_per_cell=N_PIXELS_PER_CELL, 
                        cell_per_block=N_CELLS_PER_BLOCK, 
                        hog_channel=HOG_CHANNEL, spatial_feat=SPATIAL_BINNING_FEATURES_ON, 
                        hist_feat=COLOR_HISTOGRAM_FEATURES_ON, hog_feat=HOG_FEATURES_ON)

X = np.vstack((car_features, notcar_features)).astype(np.float64)  

# Normalize the combined feature vector
X_scaler = StandardScaler().fit(X) # Fit a scaler with a feture vector
scaled_X = X_scaler.transform(X) # Apply the scaler to collected training data
print("Final size of normalized features dataset: ", scaled_X.shape)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)
print("Size of training dataset: ", X_train.shape)
print("Size of test dataset: ", X_test.shape)

#7 Train the SVC classifier using generated training set

In [ ]:
from sklearn.svm import LinearSVC
import time

# Use a linear Support Vector Classifier
svc = LinearSVC()
# Check the training time
t=time.time()
svc.fit(X_train, y_train)
print("Training duration: {} seconds".format(round(time.time()-t, 2)))
# Check the accuracy score on test dataset
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

In [ ]:
#8 Sliding windows search algorithm

In [ ]:
import cv2

def overlay_rectangles_on_image(image, rectangles, color=(0, 0, 255), thickness=6):
    out_image = np.copy(image)
    for rectangle in rectangles:
        cv2.rectangle(out_image, rectangle[0], rectangle[1], color, thickness)
    return out_image

# Define a single function that can extract features using hog sub-sampling and make predictions
def find_car_matched_windows(img, color_space, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    
    search_area = img[ystart:ystop,:,:]
    search_area_color_converted = change_color_space(search_area, color_space)
    if scale != 1:
        imshape = search_area_color_converted.shape
        search_area_color_converted = cv2.resize(search_area_color_converted, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    
    ch1 = search_area_color_converted[:,:,0]
    ch2 = search_area_color_converted[:,:,1]
    ch3 = search_area_color_converted[:,:,2]

    # Define blocks and sliding teps
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    window_size = 64 # 8 cells and 8 pix per cell
    cells_per_step = 2  # Instead of defiingn the overlap percentage, we define how many cells to shift one the next step
    
    nblocks_per_window = (window_size // pix_per_cell) - cell_per_block + 1
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire search area
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    bounding_boxes=[]
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(search_area_color_converted[ytop:ytop+window_size, xleft:xleft+window_size], (64,64))
            
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
 
            # Scale features and make a prediction
            pred_X = np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)
            test_features = X_scaler.transform(pred_X)  
            
            test_prediction = svc.predict(test_features)
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window_size*scale)
                detection_bounding_box = [(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)]
                bounding_boxes.append(detection_bounding_box)
            
    return bounding_boxes


# IMAGE FOR TESTING THE PLIPELINE
image = mpimg.imread('./test_images/bbox-example-image.jpg')

# top veritical bound for vehicle detection
y_start = 502
# bottom veritical bound for vehicle detection
y_stop = 680
# scaling factor for sliding detection window
scale = 1.1

#draw search area frame
search_area_image = cv2.rectangle(np.copy(image),(5, y_start),(image.shape[1],y_stop),(0,255,0),5) 
 
# look for initial detections using trained classifier 
matched_windows = find_car_matched_windows(image, COLOR_SPACE, y_start, y_stop, scale, svc, X_scaler,
                               N_ORIENTATIONS, N_PIXELS_PER_CELL, N_CELLS_PER_BLOCK, SPATIAL_SIZE, N_HISTOGRAM_BINS)
# draw matched search windows on test image
matched_windows_image = overlay_rectangles_on_image(image, matched_windows, (0, 0, 255), 6)
display_images_in_table([search_area_image, matched_windows_image],["Search area", "Matched windows"])

#9. Using for multiple search space configurations for improving detection accuracy

In [ ]:
# Improve coverage of bounding boxes by combining best (empirically selected) results of
# sliding window search for various permutations of scale and search areas
search_space_configs = [(463,700,0.97), (502, 680, 1.1), (504, 583, 1.06), (504, 583, 1.09), (512,548,0.54)]

matched_windows = []
image = mpimg.imread('./test_images/bbox-example-image.jpg')
search_areas_image = np.copy(image)
matched_windows_image = np.copy(image)
for search_config in search_space_configs:
    line_color = (np.random.randint(50,255),np.random.randint(50,255), np.random.randint(50,255))
    # daw search area for current search space config
    search_area = (5, search_config[0]),(image.shape[1], search_config[1])
    search_areas_image = cv2.rectangle(search_areas_image, search_area[0], search_area[1], line_color, 4)
    # look for car matched windows using trained classifier 
    matched_windows_frame = find_car_matched_windows(image, COLOR_SPACE, search_config[0], search_config[1],
                                       search_config[2], svc, X_scaler, N_ORIENTATIONS, N_PIXELS_PER_CELL, N_CELLS_PER_BLOCK, SPATIAL_SIZE, N_HISTOGRAM_BINS)
    matched_windows_image = overlay_rectangles_on_image(matched_windows_image, matched_windows_frame, line_color, 4)
    matched_windows.extend(matched_windows_frame)
    
display_images_in_table([search_areas_image, matched_windows_image],["Search areas", "Matched windows"])


In [ ]:
#10. Group matched detection windows using heatmap with filtering

In [ ]:
import numpy as np
from scipy.ndimage.measurements import label

def add_heat_to_heatmap(heatmap, bounding_boxes):
    for bounding_box in bounding_boxes:
        # Add +1 for all pixels inside each bounding box
        # Assuming each "bounding box" takes the form ((x1, y1), (x2, y2))
        heatmap[bounding_box[0][1]:bounding_box[1][1], bounding_box[0][0]:bounding_box[1][0]] += 1
    # Return updated heatmap
    return heatmap
    
def apply_threshold_to_heatmap(heatmap, threshold):
    # Zero out all pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return updated heatmap
    return heatmap

def labels_to_bounding_boxes(detected_labels, box_min_width, box_min_height):
    bounding_boxes = []
    # Iterate through all detected cars
    for car_number in range(1, detected_labels[1]+1):
        # Find pixels with each car_number label value
        nonzero_pixels = (detected_labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzero_pixels_x = np.array(nonzero_pixels[1])
        nonzero_pixels_y = np.array(nonzero_pixels[0])
        # Define a bounding box based on min/max x and y
        box_left_top = (np.min(nonzero_pixels_x), np.min(nonzero_pixels_y))
        box_right_bottom = (np.max(nonzero_pixels_x), np.max(nonzero_pixels_y))
        if(((box_right_bottom[0]-box_left_top[0]) >=box_min_width) and ((box_right_bottom[1]-box_left_top[1]) >=box_min_height)):
            bounding_boxes.append((box_left_top, box_right_bottom))
    return bounding_boxes

def get_combined_bounding_boxes(image, windows, threshold=0, box_min_width=0, box_min_height=0):
    
    heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each window in windows list
    heatmap = add_heat_to_heatmap(heatmap, windows)  
    # Apply threshold to help remove false positives
    heatmap = apply_threshold_to_heatmap(heatmap, threshold)

    # Find final boxes from heatmap using label function
    heat_labels = label(heatmap)
    return labels_to_bounding_boxes(heat_labels, box_min_width, box_min_height), heatmap

# threshold for adjusting heatmap to skip false positive windows
HEAT_THRESHOLD = 2

# additional thresholds for the size of detection bounding boxes
BOX_MIN_WIDTH=55
BOX_MIN_HEIGHT=55

image = mpimg.imread('./test_images/bbox-example-image.jpg')

# generate bounding boxes that cover all matched sliding windows
bounding_boxes, heatmap = get_combined_bounding_boxes(image, matched_windows, heat_threshold)
# draw detected car bounding boxes on test image
bounding_boxes_image = overlay_rectangles_on_image(image, bounding_boxes, (0, 255, 0), 6)
display_images_in_table([matched_windows_image, bounding_boxes_image, heatmap],["Matched sliding windows", "Combined bounding boxes", "Heatmap"])

#11. Manual tunning of search space and filtering parameters

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import IntRangeSlider
from ipywidgets import IntSlider
from ipywidgets import FloatSlider
from ipywidgets import RadioButtons

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

test_image_filenames = glob.glob('./test_images/*.jpg') 
image_selection_radio=RadioButtons(options=test_image_filenames, value=test_image_filenames[0], continuous_update=False, description="Image_file")
search_area_change_slider = IntRangeSlider(min=350, max=700, step=1, value=[390,600], continuous_update=False, description="Search area")
scale_change_slider = FloatSlider(min=0.3, max=3.5, step=0.01, value=1.5, continuous_update=False, description="Scale")
heat_threshold_change_slider = IntSlider(min=0, max=4, step=1, value=0, continuous_update=False, description="Heatmap threshold")
box_min_width_change_slider = IntSlider(min=0, max=200, step=10, value=80, continuous_update=False, description="Box min with")
box_min_height_change_slider = IntSlider(min=0, max=200, step=10, value=50, continuous_update=False, description="Box min height")


def display_search_car_matching_windows(image_file, y_threshold=(400,700), scale=1.0, heat_threshold=0, box_min_width=0, box_min_height=0):
    image = mpimg.imread(image_file)
    
    # draw search area
    search_area = (5, y_threshold[0]),(image.shape[1], y_threshold[1])
    search_area_image = cv2.rectangle(np.copy(image), search_area[0], search_area[1], (255,0,0), 3)
    
    # look for car matched windows using trained classifier 
    matched_windows_frame = find_car_matched_windows(image, COLOR_SPACE, y_threshold[0], y_threshold[1], scale,
                                                     svc, X_scaler, N_ORIENTATIONS, N_PIXELS_PER_CELL,
                                                     N_CELLS_PER_BLOCK, SPATIAL_SIZE, N_HISTOGRAM_BINS)
    matched_windows_image = overlay_rectangles_on_image(search_area_image, matched_windows_frame, (0,255,0), 3)
    
    # generate bounding boxes that cover all matched sliding windows
    bounding_boxes, heatmap = get_combined_bounding_boxes(image, matched_windows_frame, heat_threshold, box_min_width, box_min_height)
    # draw detected car bounding boxes on test image
    bounding_boxes_image = overlay_rectangles_on_image(image, bounding_boxes, (0,0,255), 6)
    
    display_images_in_table([matched_windows_image, bounding_boxes_image],
                            ["Search area Y lmits: {}-{}. Scale: {}, ".format(y_threshold[0], y_threshold[1], scale),
                             "Bounding boxes. Heatmap threshold: {}, size threshold: {}x{}".format(heat_threshold, box_min_width, box_min_height)])

interact(display_search_car_matching_windows,
         image_file = image_selection_radio,
         y_threshold = search_area_change_slider,
         scale = scale_change_slider,
         heat_threshold=heat_threshold_change_slider,
         box_min_width=box_min_width_change_slider,
         box_min_height=box_min_height_change_slider)

#12. Detecting of the bounding boxes on 'challenging' cases from test video file

In [ ]:
# search space configurations for detecting vehilces in target video file
SEARCH_SPACE_CONFIGS = [(395,488,1.32),(431,518,1.35),(405, 499, 1.20), (405, 499, 1.06), (401, 499, 0.93), (392,522,2.01), (393,510,1.83), (415,499,1.0), (389,511,1.34), (401,489,1.33), (392,608,0.9)]

def detect_vehicles(image, vis_not_filtered=False):
    matched_windows = []
    for search_config in SEARCH_SPACE_CONFIGS:
        # look for car matched windows using trained classifier 
        windows = find_car_matched_windows(image, COLOR_SPACE, search_config[0], search_config[1], search_config[2],
                                           svc, X_scaler, N_ORIENTATIONS, N_PIXELS_PER_CELL, N_CELLS_PER_BLOCK, SPATIAL_SIZE, N_HISTOGRAM_BINS)
        if(len(windows) > 0):
            matched_windows.extend(windows)
     
    bounding_boxes, heatmap =  get_combined_bounding_boxes(image, matched_windows, HEAT_THRESHOLD,BOX_MIN_WIDTH, BOX_MIN_HEIGHT)
    b_boxes_image = overlay_rectangles_on_image(image, bounding_boxes, (0,255,0), 6)
    if(vis_not_filtered == True):
        not_filtered_bounding_boxes, heatmap =  get_combined_bounding_boxes(image, matched_windows, 0,0,0)
        not_filtered_b_boxes_image = overlay_rectangles_on_image(image, not_filtered_bounding_boxes, (0,255,0), 6)
        return b_boxes_image, not_filtered_b_boxes_image
    else:
        return b_boxes_image


test_image_filenames = glob.glob('./test_images/test*.jpg') 
processed_images = []
image_labels = []
for test_image_file in test_image_filenames:
    test_image = mpimg.imread(test_image_file)
    filtered_image, not_filtered_image = detect_vehicles(test_image, vis_not_filtered=True)
    display_images_in_table([not_filtered_image, filtered_image],['not filtered detection', 'filtered detection'])

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# PROCESS VIDEO
video_in_file = './test_video.mp4'
video_out_file = './test_video_out.mp4'

video_in = VideoFileClip(video_in_file)
video_out = video_in.fl_image(detect_vehicles)
%time video_out.write_videofile(video_out_file, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_out_file))

In [ ]:
#13. Detecting a cars in target video file with multiframe detection smoothing

In [ ]:
# MULTIFRAME VEHICLE DETECTION
class MultiframeDetectionCache():
    def __init__(self, cache_size):
        self.contents = []
        self.max_size = cache_size
        
    def update(self, frame_detections):
        self.contents.append(frame_detections)
        # constantly refresh the content of the cache by cutting out 'too old' detections
        self.contents = self.contents[-self.max_size:]

def get_combined_traced_bounding_boxes(image, windows_cache, frame_threshold, box_min_width=0, box_min_height=0):
    
    heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
    # Add heat to each window in windows list
    for frame_windows in windows_cache.contents:
        # update resulting heatmap with heat from cached frame
        heatmap = add_heat_to_heatmap(heatmap, frame_windows)  
    
    # Apply the threshold to remove false positives
    heatmap = apply_threshold_to_heatmap(heatmap, frame_threshold * windows_cache.max_size)

    # Find final boxes from heatmap using label function
    heat_labels = label(heatmap)
    return labels_to_bounding_boxes(heat_labels, box_min_width, box_min_height), heatmap        
        
def detect_vehicles_traced(image):
    matched_windows = []
    for search_config in SEARCH_SPACE_CONFIGS:
        # look for car matched windows using trained classifier 
        windows = find_car_matched_windows(image, COLOR_SPACE, search_config[0], search_config[1],
                                           search_config[2], svc, X_scaler, N_ORIENTATIONS, N_PIXELS_PER_CELL,
                                           N_CELLS_PER_BLOCK, SPATIAL_SIZE, N_HISTOGRAM_BINS)
        if(len(windows) > 0):
            matched_windows.extend(windows)
            
    detection_cache.update(matched_windows)        
    
    bounding_boxes, heatmap =  get_combined_traced_bounding_boxes(image, detection_cache, HEAT_THRESHOLD, BOX_MIN_WIDTH, BOX_MIN_HEIGHT)
    b_boxes_image = overlay_rectangles_on_image(image, bounding_boxes, (0,255,0), 6)
    return b_boxes_image 


detection_cache = MultiframeDetectionCache(cache_size=15)
video_in_file = './project_video.mp4'
video_out_file = './project_video_out.mp4'

video_in = VideoFileClip(video_in_file)#.subclip(27,32) 
video_out = video_in.fl_image(detect_vehicles_traced)
%time video_out.write_videofile(video_out_file, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_out_file))